In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qutip
from scipy.linalg import expm
from scipy.constants import hbar
from scipy import signal

# Qubit evolution in the lab frame

Define the Pauli matrices

$\sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$ $\sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$ $\sigma_y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}$

In [ ]:
sigmaZ = np.array([[1,0],
                   [0,-1]])
sigmaX = np.array([[0,1],
                   [1,0]])
sigmaY = np.array([[0,-1j],
                   [1j,0]])

Define the Hamiltonian $\mathcal{H} = \frac{\hbar}{2} \omega \sigma_z$

In [ ]:
omega = 2 * np.pi

H = hbar/2 * omega * sigmaZ
print(H)

Define an initial state $| \psi_0 \rangle = c_0 \begin{pmatrix} 1 \\ 0 \end{pmatrix} + c_1 \begin{pmatrix} 0 \\ 1 \end{pmatrix}$

In [ ]:
# psi0 = np.array([1,0])
psi0 = np.array([1,1])*1/np.sqrt(2)
print(psi0)

Evolve the state in time using $| \psi_t \rangle = U(t) | \psi_0 \rangle$ with $U(t) = e^{\frac{-i \mathcal{H} t}{\hbar}}$

In [ ]:
time = np.linspace(0,0.95,96)
psiT = []

for t in time:
    psiT.append(expm(-1j*H*t/hbar) @ psi0)

Save the expectation values of x, y and z (e.g. $\langle \psi_t | \sigma_z | \psi_t \rangle$)

In [ ]:
x = [(p.conj().T @ sigmaX @ p).real for p in psiT]
y = [(p.conj().T @ sigmaY @ p).real for p in psiT]
z = [(p.conj().T @ sigmaZ @ p).real for p in psiT]

Plot the evolution of the state

In [ ]:
def qubitPlots(time,x,y,z):
    bsPlot = qutip.Bloch()
    bsPlot.add_points([x, y, z])
    bsPlot.render()
    bsPlot.fig.set_size_inches([25,5])

    ax1 = bsPlot.fig.add_subplot(133)
    ax1.plot(time,x,label="x")
    ax1.plot(time,y,label="y")
    ax1.plot(time,z,label="z")
    ax1.legend()
    ax1.set_xlabel("Time")
    ax1.set_ylabel("Expectation value")
    bsPlot.show()

In [ ]:
qubitPlots(time,x,y,z)

Let's add a drive term $\mathcal{H} = \frac{\hbar}{2} \left( \omega \sigma_z + \Omega \cos \left( \omega t \right) \sigma_x \right)$

In [ ]:
omegaStatic = 20 * np.pi
omegaDrive = 2 * np.pi

H = lambda t: hbar / 2 * (omegaStatic * sigmaZ + omegaDrive * np.cos(omegaStatic * t) * sigmaX)

With a time-dependent Hamiltonian, we need to evolve step-by-step

In [ ]:
psi0 = np.array([1,0])

time = np.linspace(0,1,1001)
dTime = time[1] - time[0]
psiT = []

for t in time:
    if t == 0:
        psiT.append(psi0)
    else:
        psiT.append(expm(-1j*H(t)*dTime/hbar) @ psiT[-1])
    
x = [(p.conj().T @ sigmaX @ p).real for p in psiT]
y = [(p.conj().T @ sigmaY @ p).real for p in psiT]
z = [(p.conj().T @ sigmaZ @ p).real for p in psiT]

In [ ]:
qubitPlots(time,x,y,z)

# Qubit evolution in the rotating frame

Define the Hamiltonian $\mathcal{H} = \frac{\hbar}{2} ( \Delta \sigma_z + \Omega_x \sigma_x + \Omega_y \sigma_y )$

In [ ]:
delta = 0 # 4 * np.pi
omegaX = 2 * np.pi
omegaY = 0 * np.pi

H = hbar/2 * ( delta*sigmaZ + omegaX*sigmaX + omegaY*sigmaY )
print(H)

Define an initial state and evolve in time

In [ ]:
psi0 = np.array([1,0])

time = np.linspace(0,0.95,96)
dTime = time[1] - time[0]
psiT = []

for t in time:
    if t == 0:
        psiT.append(psi0)
    else:
        psiT.append(expm(-1j*H*dTime/hbar) @ psiT[-1])
    
x = [(p.conj().T @ sigmaX @ p).real for p in psiT]
y = [(p.conj().T @ sigmaY @ p).real for p in psiT]
z = [(p.conj().T @ sigmaZ @ p).real for p in psiT]

In [ ]:
qubitPlots(time,x,y,z)

Let's add a detuning axis now

In [ ]:
psi0 = np.array([1,0])

time = np.linspace(0,3,301)
dTime = time[1] - time[0]

detunings = np.linspace(-4*np.pi, 4*np.pi, 51)
psiT = []
z = np.zeros([len(detunings),len(time)])

for iDet, delta in enumerate(detunings):
    H = hbar/2 * ( delta*sigmaZ + omegaX*sigmaX + omegaY*sigmaY )
    for iTime, t in enumerate(time):
        if t == 0:
            psiT.append(psi0)
        else:
            psiT.append(expm(-1j*H*dTime/hbar) @ psiT[-1])
            z[iDet, iTime] = (psiT[-1].conj().T @ sigmaZ @ psiT[-1]).real

In [ ]:
plt.pcolormesh(time, detunings, z, shading="nearest")
plt.xlabel("Time")
plt.ylabel("Detuning")

# Qubit decoherence

## Dephasing noise

Noise that modulates the qubit splitting (i.e. along the quantization axis). In most cases, this noise is pink. Let's create an array sampled from a spectrum $S_N = \frac{S_0}{\omega}$, and add it to the detuning term in the Hamiltonian.

In [ ]:
N = 1001
fs = 1000
wNoise = np.random.normal(0,50,N)
b,a = signal.butter(1, [0.01, 0.5], btype="band", fs=fs)
pNoise = signal.lfilter(b,a,wNoise)

H = lambda n: hbar/2 * (n * sigmaZ)

Start with a superposition state and evolve our noisy Hamiltonian

In [ ]:
psi0 = np.array([1,1])*1/np.sqrt(2)

time = np.linspace(0,0.95,N)
dTime = time[1] - time[0]
psiT = []

for t,n in zip(time,pNoise):
    if t == 0:
        psiT.append(psi0)
    else:
        psiT.append(expm(-1j*H(n)*dTime/hbar) @ psiT[-1])
    
x = [(p.conj().T @ sigmaX @ p).real for p in psiT]
y = [(p.conj().T @ sigmaY @ p).real for p in psiT]
z = [(p.conj().T @ sigmaZ @ p).real for p in psiT]

In [ ]:
qubitPlots(time,x,y,z)

## Characterising and decoupling dephasing noise

### Ramsey fringes - $T_2^*$

In this experiment we apply a $\pi / 2$ gate to prepare the spin on the x-y plane of the Bloch sphere, wait some time $\tau$ and apply another $\pi / 2$ gate to attempt to bring the spin back to the quantization axis.

In the simulation of this pulse sequence, we assume that we can apply a perfect $\pi / 2$ gate, and apply the unitary directly.